In [2]:
import loompy
import numpy as np
import pandas as pd
import bioservices
import pickle
from collections import Counter
import anndata

/Users/clark04/anaconda3/envs/pkgtorch/lib/python3.11/site-packages/loompy/bus_file.py:67: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
/Users/clark04/anaconda3/envs/pkgtorch/lib/python3.11/site-packages/loompy/bus_file.py:84: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
/Users/clark04/anaconda3/envs/pkgtorch/lib/python3.11/site-packages/loompy/bus_f

In [6]:
genes_table = pd.read_csv('/mnt/scratchc/ghlab/dataset_info/genes_to_symbols.csv')
genes_table.isna().sum()

X0          0
gene_ids    0
dtype: int64

In [17]:
symbol_id_dict = dict(zip(genes_table['X0'],genes_table['gene_ids']))

In [56]:
# Specify the path to your Loom file
loom_file_path = "/mnt/scratchc/ghlab/in_silico_perturb_data/Mesenchyme1_PSC1_processed_cell_types.loom"
new_file_path = "/mnt/scratchc/ghlab/in_silico_perturb_data/Mesenchyme1_PSC1_ready.loom"
# Load the Loom file
with loompy.new(new_file_path) as ds_out:
    with loompy.connect(loom_file_path,'r+') as ds:
        # Access data matrix (expression matrix)
        expression_matrix = ds[:, :]

        # Access row (gene) and column (cell) attributes
        row_attrs = pd.DataFrame(ds.ra)
        col_attrs = pd.DataFrame(ds.ca)
        #ds.ca = {attr : ["PSC1" if cluster == 2 else "Mesenchyme1" for cluster in ds.ca["seurat_clusters"]] if attr == "seurat_clusters" else ds.ca[attr] for attr in col_attrs.iloc[:,0]}
        ds.ca = {'n_counts' if attr == "nCount_RNA" else attr : ds.ca[attr] for attr in col_attrs.iloc[:,0]}
        display(ds.shape)
        #ds.ra = {'ensembl_id':np.array(gene_ids)}
        ids_present = ds.ra['ensembl_id']
        display(Counter(ds.ca["seurat_clusters"]))
        display(ds.ra['ensembl_id'])


        ds_out.add_columns(ds.layers, col_attrs=ds.ca, row_attrs=ds.ra)
            #genes = ds.ra['Gene']
    #cell_types = ["PSC1" if cluster == 3 else "Mesenchyme1" for cluster in ds.ca["seurat_clusters"]]
    
    #ds.write_loom("/Users/clark04/Mesenchyme1_PSC1_ensgid.loom")

(27041, 21341)

Counter({'PSC1': 10755, 'Mesenchyme1': 10586})

array(['ENSG00000276256', 'ENSG00000273748', 'ENSG00000271254', ...,
       'ENSG00000288049', 'ENSG00000198692', 'ENSG00000280969'],
      dtype=object)

In [35]:
ids_present

array(['ENSG00000276256', 'ENSG00000273748', 'ENSG00000271254', ...,
       'ENSG00000288049', 'ENSG00000198692', 'ENSG00000280969'],
      dtype=object)

In [48]:
Counter(a['seurat_clusters'])

Counter({'PSC1': 10755, 'Mesenchyme1': 10586})

In [43]:
col_attrs.iloc[:,0]

0                 CellID
1        SCT_snn_res.0.1
2                  cells
3       log10GenesPerUMI
4              mitoRatio
5             nCount_RNA
6             nCount_SCT
7       nCount_ambiguous
8         nCount_spliced
9       nCount_unspliced
10          nFeature_RNA
11          nFeature_SCT
12    nFeature_ambiguous
13      nFeature_spliced
14    nFeature_unspliced
15                 nGene
16                  nUMI
17                sample
18            seq_folder
19       seurat_clusters
Name: 0, dtype: object

In [7]:
pd.Series(genes).to_csv('/mnt/scratchc/ghlab/dataset_info/symbol_list.csv')

In [7]:
fail_list = []
ensembl_id = []
for gene in genes:
    try:
        ensembl_id.append(objects[0][gene])
    except KeyError:
        fail_list.append(gene)

In [ ]:
correction_dict = {"YARS":"ENSG00000134684",
                   "ATP5MD":"ENSG00000173915",
                   "C1orf195":'ENSG00000204464',
                   "UBXN10-AS1":'ENSG00000187980'}

In [18]:
gene_ids = [symbol_id_dict[gene] for gene in genes]

In [20]:
len(gene_ids)

27041